In [33]:
import os
import matplotlib.pyplot as plt
import torchvision
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils.notebook import display_scrollable_dataframe,plot_sailency
from data_loaders import CUB_extnded_dataset
from models import get_inception_transform
from IPython.display import display

from sailency import get_saliency_maps,saliency_score_part

%reload_ext autoreload
%autoreload 2

# Data analysis 
On the use of Majority voting on the concetps. First the data is loaded. 

In [23]:
# Settings for the experiment your running

# Settings for loading the dataset, make sure its the same as the one used to train the model. 
model_folder_MV = r"models\Sequential_Basemodel3"
model_folder_NoMV = r"models\Sequential_Basemodel2"


use_majority_voting_MV = True
use_majority_voting_NoMV = False

# You can find the settings in the .hydra folders config.yaml 
data_config_MV = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_MV,
                'min_class_count':10,
                'return_visibility':True}


# You can find the settings in the .hydra folders config.yaml 
data_config_NoMV = {'CUB_dir':r'data/CUB_200_2011',
                'split_file':r'data/train_test_val.pkl',
                'use_majority_voting':use_majority_voting_NoMV,
                'min_class_count':10,
                'return_visibility':True}

In [24]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

112 312


In [25]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts and visibility
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts and visibility
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [26]:
data_set = 'train' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer)
data_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer)

#Get a dataset that return original images instead normalized ones
data_human_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform)
concept_names_MV_train = data_MV_train.consept_labels_names
class_names_MV_train = data_MV_train.class_labels_names
n_classes_MV_train = data_MV_train.n_classes
n_concepts_MV_train = data_MV_train.n_concepts

data_human_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform)
concept_names_NoMV_train = data_NoMV_train.consept_labels_names
class_names_NoMV_train = data_NoMV_train.class_labels_names
n_classes_NoMV_train = data_NoMV_train.n_classes
n_concepts_NoMV_train = data_NoMV_train.n_concepts


print(n_concepts_MV_train,n_concepts_NoMV_train)

112 312


In [27]:
print(data_MV_train[0][0].shape) # image
print(data_MV_train[0][1].shape) # concepts and visibility
print(data_MV_train[0][2].shape) # classes
print(len(data_MV_train[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_train[0][0].shape) # image
print(data_NoMV_train[0][1].shape) # concepts and visibility
print(data_NoMV_train[0][2].shape) # classes
print(len(data_NoMV_train[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([112])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [34]:
data_set = 'val' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, reduce = False)
data_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, reduce = False)

#Get a dataset that return original images instead normalized ones
data_human_MV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, reduce = False)
concept_names_MV_val = data_MV_val.consept_labels_names
class_names_MV_val = data_MV_val.class_labels_names
n_classes_MV_val = data_MV_val.n_classes
n_concepts_MV_val = data_MV_val.n_concepts

data_human_NoMV_val = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, reduce = False)
concept_names_NoMV_val = data_NoMV_val.consept_labels_names
class_names_NoMV_val = data_NoMV_val.class_labels_names
n_classes_NoMV_val = data_NoMV_val.n_classes
n_concepts_NoMV_val = data_NoMV_val.n_concepts


print(n_concepts_MV_val,n_concepts_NoMV_val)

312 312


In [35]:
print(data_MV_val[0][0].shape) # image
print(data_MV_val[0][1].shape) # concepts
print(data_MV_val[0][2].shape) # classes
print(len(data_MV_val[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_val[0][0].shape) # image
print(data_NoMV_val[0][1].shape) # concepts
print(data_NoMV_val[0][2].shape) # classes
print(len(data_NoMV_val[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([312])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [36]:
data_set = 'train' # majority voting is only done  

#Define data set, the human transformer data_set is used to get the original images instead of the normalized ones
transformer = get_inception_transform(mode=data_set,methode="center")
human_tansform = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(299),torchvision.transforms.ToTensor()])

#Get dataset
data_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=transformer, reduce = False)
data_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=transformer, reduce = False)

#Get a dataset that return original images instead normalized ones
data_human_MV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_MV,transform=human_tansform, reduce = False)
concept_names_MV_train = data_MV_train.consept_labels_names
class_names_MV_train = data_MV_train.class_labels_names
n_classes_MV_train = data_MV_train.n_classes
n_concepts_MV_train = data_MV_train.n_concepts

data_human_NoMV_train = CUB_extnded_dataset(mode=data_set,config_dict=data_config_NoMV,transform=human_tansform, reduce = False)
concept_names_NoMV_train = data_NoMV_train.consept_labels_names
class_names_NoMV_train = data_NoMV_train.class_labels_names
n_classes_NoMV_train = data_NoMV_train.n_classes
n_concepts_NoMV_train = data_NoMV_train.n_concepts


print(n_concepts_MV_train,n_concepts_NoMV_train)

312 312


In [37]:
print(data_MV_train[0][0].shape) # image
print(data_MV_train[0][1].shape) # concepts
print(data_MV_train[0][2].shape) # classes
print(len(data_MV_train[0][3])) # concept coordinates, but only the mask needs to be applied

print(data_NoMV_train[0][0].shape) # image
print(data_NoMV_train[0][1].shape) # concepts
print(data_NoMV_train[0][2].shape) # classes
print(len(data_NoMV_train[0][3])) # concept coordinates, but only the mask needs to be applied

torch.Size([3, 299, 299])
torch.Size([312])
torch.Size([200])
112
torch.Size([3, 299, 299])
torch.Size([2, 312])
torch.Size([200])
312


In [38]:
# compare all the concepts in data_MV_train to data_noMV_train
diff = 0
for item_MV, item_NoMV in zip(tqdm(data_MV_val), data_NoMV_val):

    diff_i = torch.sum(item_MV[1] != item_NoMV[1][0])
    diff += diff_i

print(diff)


100%|██████████| 1198/1198 [00:26<00:00, 45.79it/s]

tensor(0)


In [39]:
# compare all the concepts in data_MV_train to data_noMV_train
diff = 0
for item_MV, item_NoMV in zip(tqdm(data_MV_train), data_NoMV_train):

    diff_i = torch.sum(item_MV[1] != item_NoMV[1][0])
    diff += diff_i

print(diff)


100%|██████████| 4796/4796 [01:17<00:00, 61.75it/s]

tensor(149144)


In [44]:
print(len(data_MV_train)*312)
print(diff/(len(data_MV_train)*312))

1496352
tensor(0.0997)


## Difference in all concepts
When changing the concepts with majority voting, is there some classes that is more affected by this change? Look at some examples of these classes. Are there some tendencies in the classes that are more affected by the change maybe the change is due to different genders in the class?


## What classes has the most non visible concepts in MV?
When doing MV you can introduce concepts into a sample that is not visible. What classes is most affected by this?